Formerly called minreverse

In [1]:
from modeling.compute import Var
from modeling.gen1.compute import Evaluable
a,b,c,d = Var('a'), Var('b'), Var('c'), Var('d')
eqlist = [(b,c+1),(a,b-2),(a,2*d+3)]
eqs = {idx: eq for idx, eq in enumerate(eqlist)}
eq = eqlist[0]
ev = Evaluable.fromsympy(eq[0]-eq[1])

In [2]:
import openmdao.api as om
from problemgen.gen1.openmdao import Impcomp

In [3]:
prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Impcomp(equation = ev, output_name='b'), promotes=['*'])
model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
model.linear_solver = om.ScipyKrylov()

prob.setup()
prob.set_val('c', 3.)
prob.run_model()

print(prob.get_val('comp.b'))

NL: Newton Converged in 1 iterations
[4.]


In [4]:
from trash.inputresolver import eqvars, default_out, getallvars, var_matched_cons, invert_edges
from graph.graphview import bipartite_repr

In [5]:
dout = default_out(eqs)
vrs = getallvars(eqs)
eqns = eqs.keys()
eqv = eqvars(eqs)
G, edges_original = bipartite_repr(eqv)
edges = invert_edges(edges_original)

In [19]:
dout

{0: b, 1: a, 2: a}

In [17]:
edges = [(0, b), (0, c), (1, a), (1, b), (2, a), (2, d)]

[(0, b), (0, c), (1, a), (1, b), (2, a), (2, d)]

In [7]:
import gurobipy as gp
from gurobipy import GRB

In [8]:
not_input = []
n_eqs = len(eqns)

In [9]:
m = gp.Model('minrevassign')
m.setParam('OutputFlag', False)
x = m.addVars(edges, name="assign", vtype=GRB.BINARY)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2527862
Academic license 2527862 - for non-commercial use only - registered to no___@mit.edu


In [21]:
edges

[(0, b), (0, c), (1, a), (1, b), (2, a), (2, d)]

In [11]:
# Matching eqs:
fcons = m.addConstrs((x.sum(j,'*') == 1 for j in eqns), name='equations')
varcons = m.addConstrs((var_matched_cons(x, j, not_input) for j in vrs), name='variables')
m.setObjective(gp.quicksum([x[(key, var)] for key,var in dout.items()]), GRB.MAXIMIZE)
# Elimination
#m.addConstr(gp.quicksum(x[(r,j)] for r,j in sol)<= n_eqs-1) 

In [12]:
m.optimize()

In [13]:
m.objVal

2.0

In [14]:
sol = [(r,j) for (r, j) in edges if x[r,j].x>1e-6]

In [15]:
sol_outset = {key:val for key,val in sol}

In [16]:
sol_outset

{0: b, 1: a, 2: d}